In [ ]:
cd /content/drive/Shareddrives/과학기술일자리진흥원_은채/200901_210228_인턴/지식재산연구원 논문/동향분석(딥러닝)/기업정보연계

/content/drive/Shareddrives/과학기술일자리진흥원_은채/200901_210228_인턴/지식재산연구원 논문/동향분석(딥러닝)/기업정보연계


# KSIC 코드로 정리

In [ ]:
import pandas as pd

In [ ]:
ksic9 = pd.read_csv('KSIC9차_KSIC코드.csv')

In [ ]:
import re
p = re.compile('\S+\(') 
set(ksic9['산업분류(KSIC)'].apply(lambda x: len(p.findall(x)))) # 여러 개의 괄호가 있는 건 없다.

{1}

In [ ]:
ksic9['산업분류(KSIC)_중'] = ksic9['산업분류(KSIC)'].apply(lambda x: x.split('(')[0])

In [ ]:
ksic9['산업분류(KSIC)_중'] = ksic9['산업분류(KSIC)_중'].apply(lambda x: x.split(', '))

In [ ]:
import re
p = re.compile('\S+\(\S+\)')
ksic9['산업분류(KSIC)_세1'] = ksic9['산업분류(KSIC)'].apply(lambda x: p.findall(x)[0].split('(')[0])
ksic9['산업분류(KSIC)_세2'] = ksic9['산업분류(KSIC)'].apply(lambda x: p.findall(x)[0].replace(')','').split('(')[1])

In [ ]:
ksic9

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
0,농림어업,"A01, A02, A03(A0000)","[A01, A02, A03]",A03,A0000
1,광업,"B05, B06, B07, B08(B0000)","[B05, B06, B07, B08]",B08,B0000
2,식료품 제조업,C10(C1000),[C10],C10,C1000
3,음료제조업,C11(C1100),[C11],C11,C1100
4,담배제조업,C12(C1200),[C12],C12,C1200
...,...,...,...,...,...
56,기타제품 제조업,C33(C3300),[C33],C33,C3300
57,"전기, 가스, 증기, 수도","D35, D36(D0000)","[D35, D36]",D36,D0000
58,"하수, 폐기물처리, 원료재생, 환경복원","E37, E38, E39(E0000)","[E37, E38, E39]",E39,E0000
59,"건설업, 건축기술 서비스","F41, F42(Y0000)","[F41, F42]",F42,Y0000


In [ ]:
ksic9 = ksic9.explode('산업분류(KSIC)_중')

In [ ]:
ksic9[ksic9['산업분류(KSIC)_중']=='C20']

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
12,기초 화학물질제조업,C20(C2010),C20,C20,C2010
13,비료 및 질소 화학물 제조업,C20(C2020),C20,C20,C2020
14,합성고무 및\n플라스틱 물질 제조업,C20(C2030),C20,C20,C2030
15,살충제 및 기타농약제조업,C20(C2041),C20,C20,C2041
16,"잉크, 페인트 코팅제 및 유사제품 제조업",C20(C2042),C20,C20,C2042
17,"세제, 화장품 및 광택제 제조업",C20(C2043),C20,C20,C2043
18,그 외 기타 화학제품 제조업,C20(C2049),C20,C20,C2049
19,화학섬유 제조업,C20(C2050),C20,C20,C2050


In [ ]:
ksic9

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
0,농림어업,"A01, A02, A03(A0000)",A01,A03,A0000
0,농림어업,"A01, A02, A03(A0000)",A02,A03,A0000
0,농림어업,"A01, A02, A03(A0000)",A03,A03,A0000
1,광업,"B05, B06, B07, B08(B0000)",B05,B08,B0000
1,광업,"B05, B06, B07, B08(B0000)",B06,B08,B0000
...,...,...,...,...,...
59,"건설업, 건축기술 서비스","F41, F42(Y0000)",F41,F42,Y0000
59,"건설업, 건축기술 서비스","F41, F42(Y0000)",F42,F42,Y0000
60,"컴퓨터 프로그래밍, 정보서비스업","J582, J62, J63(Z0000)",J582,J63,Z0000
60,"컴퓨터 프로그래밍, 정보서비스업","J582, J62, J63(Z0000)",J62,J63,Z0000


In [ ]:
ksic9.reset_index(drop=True, inplace=True)
ksic9['KSIC_CODE'] = ksic9['산업분류(KSIC)_중']
ksic9.loc[ksic9.duplicated('산업분류(KSIC)_중', keep=False), 'KSIC_CODE'] = ksic9.loc[ksic9.duplicated('산업분류(KSIC)_중', keep=False), '산업분류(KSIC)_세2']

In [ ]:
ksic9 = ksic9[['산업명', 'KSIC_CODE']]

# KSIC -> IPC

In [ ]:
import pandas as pd

- table 정제

In [ ]:
table = pd.read_csv('기업정보연계+KSIC_IPC_연계_final.CSV', encoding='cp949')

In [ ]:
table

,농림어업,광업,식료품 제조업,음료제조업,담배제조업,섬유제품 제조업\n:의복제외,"의복, 의복 액세서리 및 모피제품 제조업","가죽, 가방 및 신발제조업",목재 및\n나무제품제조업; 가구제외,"펄프, 종이 및\n종이제품 제조업",인쇄 및\n기록매체복제업,"코크스, 연탄 및\n석유정제품 제조업",기초 화학물질제조업,비료 및 질소 화학물 제조업,합성고무 및\n플라스틱 물질 제조업,살충제 및 기타농약제조업,"잉크, 페인트 코팅제 및 유사제품 제조업","세제, 화장품 및 광택제 제조업",그 외 기타 화학제품 제조업,화학섬유 제조업,의료용 물질 및 의약품 제조업,고무제품 및\n플라스틱 제품 제조업,비금속광물제품 제조업,1차 철강제조업,1차 비철금속 제조업,금속주조업,"구조용 금속제품,\n탱크, 증기발생기 제조업",무기 및 총포탄 제조업,기타 금속가공 제품 제조업,반도체 제조업,전자부품 제조업,컴퓨터 및 주변장치 제조업,통신 및 방송 장비 제조업,영상 및 음향기기 제조업,마그네틱 및 광학 매체 제조업,의료용 기기 제조업,"측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업","안경, 사진장비 및\n기타 광학기기 제조업",시계 및 시계부품 제조업,"전동기, 발전기 및\n전기변환, 공급, 제어장치 제조업",일차전지 및 축전지 제조업,절연선 및 케이블제조업,전구 및 조명장치 제조업,가정용 기기 제조업,기타 전기장비 제조업,사무용 기계 및\n장비 제조업,사무용 이외의 일반기계 제조업,특수 기계제조업,자동차 제조업,선박 제조업,철도장비 제조업,항공기 제조업,전투용 차량 제조업,모터사이클 제조업,그 외 분류 안된\n운송장비 제조업,가구 제조업,기타제품 제조업,"전기, 가스, 증기, 수도","하수, 폐기물처리, 원료재생, 환경복원","건설업, 건축기술 서비스","컴퓨터 프로그래밍, 정보서비스업"
0,A01B27,C22B,A21D,C12C,A24B,D04D,A41B,A43B,A44C23,B42F,B41M,C10G,C01C,C05B,C08B,A01N,C09D,A61Q,A62D,D01C,A61P,B29B,A47K,B21C,C22F,B22D,A47H,F41A,A01L,G11C,B05D,B41J2,G03H,G03B31,G11B,A61B,G01B,G02B,G04B,F99Z,H01M,H01B,B60Q,A45D,B60M,B41L,B01B,A01B(A01B27/02 제외),B60B,B60V,B60L13,B64B,F41H7,B62J,B62B,A47B,A41G,F02C,B09B,E01C,G06D
1,A01B02,E21D,A23B,C12F,A24D,D04G,A41C,A43C,B27D,D21C,B42D,C10L,C01D,C05C,C08F,A01P,NaN,C09F,B01J,D01D,C07D,B29C,B24D,C21B,C25C,NaN,B21G,F41B,A44B11,H01L,B81B,B41J27,H01P,G10L,NaN,A61C,G01C,G02C,G04C,H02B,NaN,H01H,F21H,A47G,C25F,B41J1,B01D,A01F,B60D,B63B,B61B,B64C,NaN,B62K11,B62C,A47C,A44B(A44B11 제외),F03D,B09C,E01D,G06E
2,A01C,NaN,A23C,C12G,A24F,D04H,A41D,A45C,B27K,D21H,NaN,NaN,C01F,C05D,C08G,NaN,NaN,C09G,C06B,D01F,C07J,B29D,B28B,C21C,G21H(G21H1 제외),NaN,F16T,F41C,B21D,NaN,B81C,B41J29,H01Q,H03J,NaN,A61D,G01D,G03D,G04D,H02J,NaN,H01R,F21K,A47J,G08B,G03G,B04C,A01J,B60F,B63C,B61C,B64D,NaN,B62M6,B62H,A47D,A44C(A44C23제외),H02S,B65F,E01F,G06F(G06F1 제외)
3,A01D,NaN,A23D,C12H,NaN,D06C,A41F,B68B,B27H,D21J,NaN,NaN,C01G,C05F,C08J,NaN,NaN,C11D,C06C,NaN,C07K,B29K,B28C,C21D,NaN,NaN,F17B,F41F,B21F,NaN,B82B,B41J31,H01S,H04H,NaN,A61F,G01F,G02F2,G04F,H02K,NaN,H02G,F21L,A47L,G08G,G06C,B05B,A01M,B60G,B63G,B61D,B64F,NaN,B62M7,B62K(B62K11 제외),A47F,A45B,G21H1,C02F,E02B,G06F(G06F3 제외)
4,A01G,NaN,A23F,NaN,NaN,D06J,A42B(A42B3 제외),B68C,B27M,NaN,NaN,NaN,C07B,C05G,C08K,NaN,NaN,D06L,C06D,NaN,C12N,B29L,B28D,C22C,NaN,NaN,F22B,F41G,B21H,NaN,B82Y,B41J32,H03B,H04N,NaN,A61G,G01H,G02F3,G04G,H02N,NaN,NaN,F21S,D06F,G10K,G06M,B23F,A21B,B60H,B63H,B61F,B64G,NaN,NaN,B62L,NaN,A45F,NaN,G21F,E02D,G06G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F02F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F26B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
table.columns = pd.Series(table.columns).str.replace('\n', ' ')

In [ ]:
table['특수 기계제조업']

0     A01B(A01B27/02 제외)
1                   A01F
2                   A01J
3                   A01M
4                   A21B
             ...        
92                  F02F
93                  F16N
94                  F16P
95                  F26B
96                  H05H
Name: 특수 기계제조업, Length: 97, dtype: object

In [ ]:
table.columns

Index(['농림어업', '광업', '식료품 제조업', '음료제조업', '담배제조업', '섬유제품 제조업 :의복제외',
       '의복, 의복 액세서리 및 모피제품 제조업', '가죽, 가방 및 신발제조업', '목재 및 나무제품제조업; 가구제외',
       '펄프, 종이 및 종이제품 제조업', '인쇄 및 기록매체복제업', '코크스, 연탄 및 석유정제품 제조업',
       '기초 화학물질제조업', '비료 및 질소 화학물 제조업', '합성고무 및 플라스틱 물질 제조업', '살충제 및 기타농약제조업',
       '잉크, 페인트 코팅제 및 유사제품 제조업', '세제, 화장품 및 광택제 제조업', '그 외 기타 화학제품 제조업',
       '화학섬유 제조업', '의료용 물질 및 의약품 제조업', '고무제품 및 플라스틱 제품 제조업', '비금속광물제품 제조업',
       '1차 철강제조업', '1차 비철금속 제조업', '금속주조업', '구조용 금속제품, 탱크, 증기발생기 제조업',
       '무기 및 총포탄 제조업', '기타 금속가공 제품 제조업', '반도체 제조업', '전자부품 제조업',
       '컴퓨터 및 주변장치 제조업', '통신 및 방송 장비 제조업', '영상 및 음향기기 제조업', '마그네틱 및 광학 매체 제조업',
       '의료용 기기 제조업', '측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업', '안경, 사진장비 및 기타 광학기기 제조업',
       '시계 및 시계부품 제조업', '전동기, 발전기 및 전기변환, 공급, 제어장치 제조업', '일차전지 및 축전지 제조업',
       '절연선 및 케이블제조업', '전구 및 조명장치 제조업', '가정용 기기 제조업', '기타 전기장비 제조업',
       '사무용 기계 및 장비 제조업', '사무용 이외의 일반기계 제조업', '특수 기계제조업', '자동차 제조업', '선박 제조업',
       '철도장비 제조업', '항공기 제조업', '전투용 차량 제조업', '모터

- query 추출

In [ ]:
def change_format(x):
    x = x.strip()
    if len(x) == 4:
        x = x + '*'
    else:
        if '/' not in x:
            x = x[:4] + '-' + x[4:].zfill(3) + '*'
        else:
            x2 = x.split('/')
            x = x2[0][:4] + '-' + x2[0][4:].zfill(3) + '/' + x2[1]
    return x

In [ ]:
def make_query(i):
    result = []
    exceptions = list(filter(lambda x: '제외' in x,table[table.columns[i]][table[table.columns[i]].notnull()])) 
    # exception 있는 코드 제외하고 나열
    for x in table[table.columns[i]][table[table.columns[i]].notnull()]:
        x = x.strip()
        if x in exceptions:
            continue
        else:
            x = change_format(x)
            result.append(x)
    # exception 있는 경우에 대해
    keys = [change_format(e.split('(')[0]) for e in exceptions]
    exc_dict = {k: [] for k in keys}
    for e in exceptions:
        exc_dict[change_format(e.split('(')[0])].append(change_format(e.split('(')[1].replace(')','').replace('제외','')))
    for k, v in exc_dict.items():
        result.append('({} not ({}))'.format(k, ' or '.join(v)))
    # 최종 query
    return 'IC = ({})'.format(' or '.join(result))

In [ ]:
from tqdm import tqdm

queries = {}
for i, column in tqdm(enumerate(table.columns)):
    query = make_query(i)
    queries[column] = query

61it [00:00, 1225.05it/s]


In [ ]:
queries

{'1차 비철금속 제조업': 'IC = (C22F* or C25C* or G21H-(G21H1 제외)* or (G21H* not (G21H-001*)))',
 '1차 철강제조업': 'IC = (B21C* or C21B* or C21C* or C21D* or C22C* or F16S* or F17C*)',
 '가구 제조업': 'IC = (A47B* or A47C* or A47D* or A47F*)',
 '가정용 기기 제조업': 'IC = (A45D* or A47G* or A47J* or A47L* or D06F* or E06C* or F24B* or F24C* or H05B*)',
 '가죽, 가방 및 신발제조업': 'IC = (A43B* or A43C* or A45C* or B68B* or B68C*)',
 '건설업, 건축기술 서비스': 'IC = (E01C* or E01D* or E01F* or E02B* or E02D* or E03B* or E03C* or E03F* or E04H* or E05B* or E05C* or E06B* or E21C* or E99Z* or F42D*)',
 '고무제품 및 플라스틱 제품 제조업': 'IC = (B29B* or B29C* or B29D* or B29K* or B29L* or A42B-003* or B60C* or B65D* or B67D* or C08C* or F16L*)',
 '광업': 'IC = (C22B* or E21D*)',
 '구조용 금속제품, 탱크, 증기발생기 제조업': 'IC = (A47H* or B21G* or F16T* or F17B* or F22B* or F22G* or F24D* or F24H* or G21B* or G21C* or G21D*)',
 '그 외 기타 화학제품 제조업': 'IC = (A62D* or B01J* or C06B* or C06C* or C06D* or C06F* or C08H* or C09H* or C09J* or C09K* or C10M* or C10N* or C11B* o

# KSIC, 국세청코드 - IPC

In [ ]:
import pandas as pd

In [ ]:
concordance = pd.read_csv('국세청업종코드-KSIC10-KSIC9_연계완.csv')

In [ ]:
concordance.head()

,일련번호,2018년 귀속(현행)업종코드,2019년 귀속(개편후)업종코드,국세청_대분류코드,국세청_대분류,국세청_중분류코드,국세청_중분류,국세청_소분류코드,국세청_소분류,국세청_세분류코드,국세청_세분류,국세청_세세분류,국세청_연계,KSIC10_코드,KSIC10_대분류코드,KSIC10_대분류,KSIC10_중분류코드,KSIC10_중분류,KSIC10_소분류코드,KSIC10_소분류,KSIC10_세분류코드,KSIC10_세분류,KSIC10_세세분류,KSIC10_연계,메인,세부설명,KSIC10_항목명,KSIC9_코드,KSIC9_항목명
0,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,01110,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,곡물 및 기타 식량 작물 재배업,1110.0,곡물 및 기타 식량 작물 재배업
1,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,01110,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,,1140.0,기타 작물 재배업
2,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,채소작물 재배업,1121.0,채소작물 재배업
3,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1131.0,과실작물 재배업
4,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1132.0,음료용 및 향신용 작물 재배업


In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               2
KSIC10_대분류코드            2
KSIC10_대분류              0
KSIC10_중분류코드            2
KSIC10_중분류              0
KSIC10_소분류코드            2
KSIC10_소분류              0
KSIC10_세분류코드            2
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1592
세부설명                 1296
KSIC10_항목명             82
KSIC9_코드               63
KSIC9_항목명              63
dtype: int64

In [ ]:
concordance = concordance[~concordance['KSIC10_코드'].isna()]

- 중분류 기준 병합

In [ ]:
concordance['KSIC_CODE'] = concordance['KSIC10_대분류코드']+concordance['KSIC10_중분류코드'].apply(lambda x: str(int(x)).zfill(2))

In [ ]:
concordance['KSIC_CODE']

0       A01
1       A01
2       A01
3       A01
4       A01
       ... 
1997    U99
1998    M71
1999    M71
2000    T98
2001    T98
Name: KSIC_CODE, Length: 2000, dtype: object

In [ ]:
concordance = concordance.merge(ksic9, how='left')

In [ ]:
concordance

,일련번호,2018년 귀속(현행)업종코드,2019년 귀속(개편후)업종코드,국세청_대분류코드,국세청_대분류,국세청_중분류코드,국세청_중분류,국세청_소분류코드,국세청_소분류,국세청_세분류코드,국세청_세분류,국세청_세세분류,국세청_연계,KSIC10_코드,KSIC10_대분류코드,KSIC10_대분류,KSIC10_중분류코드,KSIC10_중분류,KSIC10_소분류코드,KSIC10_소분류,KSIC10_세분류코드,KSIC10_세분류,KSIC10_세세분류,KSIC10_연계,메인,세부설명,KSIC10_항목명,KSIC9_코드,KSIC9_항목명,KSIC_CODE,산업명
0,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,01110,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,곡물 및 기타 식량 작물 재배업,1110.0,곡물 및 기타 식량 작물 재배업,A01,농림어업
1,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,01110,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,,1140.0,기타 작물 재배업,A01,농림어업
2,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,채소작물 재배업,1121.0,채소작물 재배업,A01,농림어업
3,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1131.0,과실작물 재배업,A01,농림어업
4,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,01121,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1132.0,음료용 및 향신용 작물 재배업,A01,농림어업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1589.0,NaN,990090.0,U,국제 및 외국기관,99.0,국제 및 외국기관,990.0,국제 및 외국기관,9900.0,국제 및 외국기관,기타 국제 및 외국기관,1.0,99009,U,국제 및 외국기관,99.0,국제 및 외국기관,990.0,국제 및 외국기관,9900.0,국제 및 외국기관,기타 국제 및 외국기관,1,NaN,NaN,기타 국제 및 외국기관,99009.0,기타 국제 및 외국기관,U99,NaN
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71511,M,"전문, 과학 및 기술 서비스업",71.0,전문 서비스업,715.0,회사본부 및 경영 컨설팅 서비스업,7151.0,회사본부,제조업 회사본부,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,제조업 회사본부,71511.0,제조업 회사본부,M71,NaN
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71519,M,"전문, 과학 및 기술 서비스업",71.0,전문 서비스업,715.0,회사본부 및 경영 컨설팅 서비스업,7151.0,회사본부,기타 산업 회사본부,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,기타 산업 회사본부,71519.0,기타 산업 회사본부,M71,NaN
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98100,T,가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,98.0,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981.0,자가 소비를 위한 가사 생산 활동,9810.0,자가 소비를 위한 가사 생산 활동,자가 소비를 위한 가사 생산 활동,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,자가 소비를 위한 가사 생산 활동,98100.0,자가 소비를 위한 가사 생산 활동,T98,NaN


In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1590
세부설명                 1296
KSIC10_항목명             80
KSIC9_코드               61
KSIC9_항목명              61
KSIC_CODE               0
산업명                  1407
dtype: int64

In [ ]:
concordance.loc[concordance['산업명'].isna(), 'KSIC_CODE'].unique()

array(['C20', 'C26', 'C24', 'C25', 'C29', 'C28', 'C27', 'C31', 'N75',
       'C34', 'E36', 'L68', 'G45', 'G47', 'G46', 'H49', 'H52', 'H50',
       'H51', 'I55', 'I56', 'J58', 'J59', 'J61', 'J60', 'K64', 'K65',
       'K66', 'M70', 'M71', 'M72', 'M73', 'N76', 'N74', 'O84', 'P85',
       'Q86', 'Q87', 'R91', 'R90', 'S96', 'S95', 'S94', 'T97', 'U99',
       'T98'], dtype=object)

- 소분류 기준 병합

In [ ]:
concordance.loc[concordance['산업명'].isna(), 'KSIC_CODE'] = concordance.loc[concordance['산업명'].isna(), 'KSIC10_대분류코드']+concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류코드'].apply(lambda x: str(int(x)).zfill(3))

In [ ]:
(concordance['KSIC_CODE'] == 'J582').sum()

5

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = concordance.loc[concordance['산업명'].isna(),concordance.columns[:-1]].merge(ksic9, how='left')['산업명']

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1590
세부설명                 1296
KSIC10_항목명             80
KSIC9_코드               61
KSIC9_항목명              61
KSIC_CODE               0
산업명                  1402
dtype: int64

- 세분류 기준

In [ ]:
concordance['KSIC9_코드'].isna().sum()

61

In [ ]:
for i in concordance.loc[concordance['산업명'].isna()].index:
    if pd.notnull(concordance.loc[i, 'KSIC9_코드']):
        concordance.loc[i, 'KSIC_CODE'] = concordance.loc[i, 'KSIC10_대분류코드']+str(int(concordance.loc[i, 'KSIC9_코드'])).zfill(4)

In [ ]:
set(ksic9['KSIC_CODE'])

{'A01',
 'A02',
 'A03',
 'B05',
 'B06',
 'B07',
 'B08',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C2010',
 'C2020',
 'C2030',
 'C2041',
 'C2042',
 'C2043',
 'C2049',
 'C2050',
 'C21',
 'C22',
 'C23',
 'C2410',
 'C2420',
 'C2430',
 'C2510',
 'C2520',
 'C2590',
 'C2610',
 'C2620',
 'C2630',
 'C2640',
 'C2650',
 'C2660',
 'C2710',
 'C2720',
 'C2730',
 'C2740',
 'C2810',
 'C2820',
 'C2830',
 'C2840',
 'C2850',
 'C2890',
 'C2918',
 'C2919',
 'C2920',
 'C30',
 'C3110',
 'C3120',
 'C3130',
 'C3191',
 'C3192',
 'C3199',
 'C32',
 'C33',
 'D35',
 'D36',
 'E37',
 'E38',
 'E39',
 'F41',
 'F42',
 'J582',
 'J62',
 'J63'}

In [ ]:
concordance['KSIC_CODE']

0          A01
1          A01
2          A01
3          A01
4          A01
         ...  
1995    U99009
1996    M71511
1997    M71519
1998    T98100
1999    T98200
Name: KSIC_CODE, Length: 2000, dtype: object

In [ ]:
(concordance['KSIC_CODE'] == 'C2010').sum()

0

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = concordance.loc[concordance['산업명'].isna(),concordance.columns[:-1]].merge(ksic9, how='left')['산업명']

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1590
세부설명                 1296
KSIC10_항목명             80
KSIC9_코드               61
KSIC9_항목명              61
KSIC_CODE               0
산업명                  1402
dtype: int64

- 이름 기준 병합

In [ ]:
for k in queries.keys():
    if k in concordance['KSIC9_항목명']:
        code = k

,KSIC10_소분류,국세청_소분류,KSIC9_항목명,KSIC10_코드
0,작물 재배업,작물 재배업,곡물 및 기타 식량 작물 재배업,01110
1,작물 재배업,작물 재배업,기타 작물 재배업,01110
2,작물 재배업,작물 재배업,채소작물 재배업,01121
3,작물 재배업,작물 재배업,과실작물 재배업,01121
4,작물 재배업,작물 재배업,음료용 및 향신용 작물 재배업,01121
...,...,...,...,...
1997,국제 및 외국기관,국제 및 외국기관,기타 국제 및 외국기관,99009
1998,회사본부 및 경영 컨설팅 서비스업,NaN,제조업 회사본부,71511
1999,회사본부 및 경영 컨설팅 서비스업,NaN,기타 산업 회사본부,71519
2000,자가 소비를 위한 가사 생산 활동,NaN,자가 소비를 위한 가사 생산 활동,98100
